<a href="https://colab.research.google.com/github/Adhambarakat/supreme-chainsaw/blob/main/regressionXiot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [16]:
!pip install influxdb-client
!pip install flightsql-dbapi
!pip install pandas
!pip3 install paho-mqtt

Importing Libraries

In [17]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

import time
import paho.mqtt.client as paho
from paho import mqtt

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.engine.training import optimizer
from google.colab import files
import matplotlib.pyplot as plt
import keras.utils as to_categorica

Loading Data

In [18]:
path = files.upload()

Saving ppgsbpdbp.xlsx to ppgsbpdbp.xlsx


Allocating Data

In [19]:
data = pd.read_excel('/content/ppgsbpdbp.xlsx')
input_dim=1
x_train = data['PPG']  #ptt training
systolic = data['SBP']  #systolic
diastolic = data['DBP'] #diastolic
y_train = np.column_stack((systolic,diastolic))

Defining the Model

In [20]:
#defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2,activation='relu', input_shape=[input_dim])
])

Compiling the Model

In [21]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])

Training the Model

In [22]:
model.fit(x_train,y_train, epochs=10, batch_size=10)

Epoch 1/10
104858/104858 [==============================] - 193s 2ms/step - loss: 3494.7495 - mae: 44.4273
Epoch 2/10
104858/104858 [==============================] - 184s 2ms/step - loss: 181.4984 - mae: 10.4085
Epoch 3/10
104858/104858 [==============================] - 184s 2ms/step - loss: 161.6592 - mae: 9.8496
Epoch 4/10
104858/104858 [==============================] - 185s 2ms/step - loss: 161.6611 - mae: 9.8498
Epoch 5/10
104858/104858 [==============================] - 187s 2ms/step - loss: 161.6604 - mae: 9.8497
Epoch 6/10
104858/104858 [==============================] - 181s 2ms/step - loss: 161.6591 - mae: 9.8492
Epoch 7/10
104858/104858 [==============================] - 187s 2ms/step - loss: 161.6611 - mae: 9.8499
Epoch 8/10
104858/104858 [==============================] - 184s 2ms/step - loss: 161.6579 - mae: 9.8497
Epoch 9/10
104858/104858 [==============================] - 185s 2ms/step - loss: 161.6598 - mae: 9.8497
Epoch 10/10
104858/104858 [=========================

Connecting to Influxdb

In [23]:
# !export INFLUXDB_TOKEN=-g6iC2cK-mwZpYCPaiKdDOyyX5P1vkncIDSigkxs_ImaQJUDUm1-87FX4peKEmW4F2DC9CMSqZKVVTLNvXERSg==

#token = os.environ.get("INFLUXDB_TOKEN")
token = "-g6iC2cK-mwZpYCPaiKdDOyyX5P1vkncIDSigkxs_ImaQJUDUm1-87FX4peKEmW4F2DC9CMSqZKVVTLNvXERSg=="
org = "test"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

x = 123
print (x)

123


Receiving and Sending Data

In [72]:
# setting callbacks for different events to see if it works, print the message etc.
def on_connect(client, userdata, flags, rc, properties=None):
    print("CONNACK received with code %s." % rc)

# with this callback you can see if your publish was successful
def on_publish(client, userdata, mid, properties=None):
    print("mid: " + str(mid))

# print which topic was subscribed to
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

# print message, useful for checking if it was successful
def on_message(client, userdata, msg):
    #print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    #print(int(msg.payload))
    if msg.topic == "heartrate" :
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point1": {
          "location": "Hospital",
          "dataa": "hr",
          "valuee": int(msg.payload),
        },
      }
      for key in data:
        point = (
          Point("hr")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(msg.payload)
    else :
      x_new=[[float(msg.payload)]]
      y_new = model.predict(np.array([[x_new]]))
      a = y_new[0]
      b = a[0]
      c = b[0]
      sbp = int(c[0])
      dbp = int(c[1])
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point2": {
          "location": "Hospital",
          "dataa": "sbp",
          "valuee": sbp,
        },
      }
      for key in data:
        point = (
          Point("sp")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point3": {
          "location": "Hospital",
          "dataa": "dbp",
          "valuee": dbp,
        },
      }
      for key in data:
        point = (
          Point("dp")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(sbp)
      print(dbp)


    """
    bucket="colabtest"
    # Define the write api
    write_api = write_client.write_api(write_options=SYNCHRONOUS)
    data = {
      "point4": {
        "location": "Hospital",
        "dataa": "hr",
        "valuee": int(msg.payload),
      },
    }
    for key in data:
      point = (
        Point("bps")
        .tag("location", data[key]["location"])
        .field(data[key]["dataa"], data[key]["valuee"])
      )
    write_api.write(bucket=bucket, org=org, record=point)
    time.sleep(1) # separate points by 1 second
    print("Complete. Return to the InfluxDB UI.")
    """



# using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
# userdata is user defined data of any type, updated by user_data_set()
# client_id is the given name of the client
client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect

# enable TLS for secure connection
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
# set username and password
client.username_pw_set("ahmed", "Ahmed1603")
# connect to HiveMQ Cloud on port 8883 (default for MQTT)
client.connect("7ea331cfad864375a4a189652c9375e0.s1.eu.hivemq.cloud", 8883)

# setting callbacks, use separate functions like above for better visibility
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_publish = on_publish

# subscribe to all topics of encyclopedia by using the wildcard "#"
client.subscribe("heartrate", qos=1)
client.subscribe("getgreen", qos=1)

# a single publish, this can also be done in loops, etc.
# client.publish("encyclopedia/temperature", payload="hot", qos=1)

# loop_forever for simplicity, here you need to stop the loop manually
# you can also use loop_start and loop_stop
client.loop_forever()

CONNACK received with code Success.
Subscribed: 1 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7fc93d6dc7c0>]
Subscribed: 2 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7fc93d6de740>]
Complete. Return to the InfluxDB UI.
b'73'
1/1 [==============================] - 0s 32ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
125
49
Complete. Return to the InfluxDB UI.
b'73'
1/1 [==============================] - 0s 24ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
125
49
Complete. Return to the InfluxDB UI.
b'73'
1/1 [==============================] - 0s 22ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
125
49
Complete. Return to the InfluxDB UI.
b'73'
1/1 [==============================] - 0s 26ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
125
49
Complete. Return to the InfluxDB UI.
b'73'
1/1 [==============================] - 0s 37ms/step
Complete. R

KeyboardInterrupt: ignored